In [39]:
!pip install numpy yfinance pandas matplotlib xgboost nltk --upgrade scipy scikit-learn tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl.metadata (4.6 kB)
Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl (332.0 MB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\prana\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python313\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\fault_injection\\fault_injection_service_config_parser.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\prana\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [40]:
import os
import requests
import time
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import yfinance as yf
from pathlib import Path
import numpy as np
import joblib
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cdist
from sklearn.metrics import accuracy_score
import xgboost as xgb

In [41]:
TOP_50_TICKERS = [
    "NVDA", "AAPL", "GOOG", "MSFT", "AMZN", "META", "TSLA", "AVGO", "2222.SR", "TSM",
    "BRK-B", "LLY", "WMT", "JPM", "TCEHY", "V", "ORCL", "MA", "005930.KS", "XOM",
    "JNJ", "PLTR", "BAC", "ASML", "ABBV", "NFLX", "601288.SS", "COST", "MC.PA", "BABA",
    "1398.HK", "AMD", "HD", "601939.SS", "ROG.SW", "PG", "GE", "MU", "CSCO", "KO",
    "WFC", "CVX", "UNH", "MS", "SAP", "TM", "AZN", "IBM", "CAT", "000660.KS"
]

current_dir = Path.cwd()
project_root = current_dir if current_dir.name != 'Notebooks' else current_dir.parent
raw_data_path = project_root / "Data" 
raw_data_path.mkdir(parents=True, exist_ok=True)

def fetch_tickers_in_batches(tickers, batch_size=10, period="6mo"):
    all_data = []
    for i in range(0, len(tickers), batch_size):
        batch = tickers[i:i + batch_size]
        print(f"Downloading batch: {batch}")
        try:
            data = yf.download(batch, period=period, group_by='ticker', auto_adjust=True, threads=True)
            if not data.empty:
                all_data.append(data)
        except Exception as e:
            print(f"Batch download error: {e}")
        time.sleep(1)
    return pd.concat(all_data, axis=1) if all_data else pd.DataFrame()

def robust_downloader(tickers, period="6mo", max_retries=3):
    print(f"Initiating resilient download for {len(tickers)} tickers...")
    df = fetch_tickers_in_batches(tickers, batch_size=15, period=period)
    
    for attempt in range(max_retries):
        existing_tickers = df.columns.get_level_values(0).unique().tolist()
        failed_tickers = [t for t in tickers if t not in existing_tickers]
        
        if not failed_tickers:
            print("All tickers downloaded successfully.")
            break
            
        wait_time = (attempt + 1) * 5
        print(f"Attempt {attempt + 1}/{max_retries}: {len(failed_tickers)} failures. Retrying in {wait_time}s...")
        time.sleep(wait_time)
        
        for ticker in failed_tickers:
            try:
                retry_data = yf.download(ticker, period=period, auto_adjust=True, progress=False)
                if not retry_data.empty and len(retry_data) > 20:
                    # Fix MultiIndex issue
                    if not isinstance(retry_data.columns, pd.MultiIndex):
                        retry_data.columns = pd.MultiIndex.from_product([[ticker], retry_data.columns])
                    df = pd.concat([df, retry_data], axis=1)
                    print(f"✓ Successfully retrieved {ticker}")
            except Exception as e:
                print(f"✗ Failed for {ticker}: {str(e)[:50]}")

    df = df.sort_index().interpolate(method='time').ffill().bfill()
    
    missing_final = [t for t in tickers if t not in df.columns.get_level_values(0).unique()]
    if missing_final:
        print(f"⚠ CRITICAL: Still missing {len(missing_final)} tickers: {missing_final}")
    
    file_path = raw_data_path / "market_data_raw.csv"
    df.to_csv(file_path)
    print(f"✓ Data ingestion complete. File saved: {file_path}")
    print(f"✓ Successfully collected {df.columns.get_level_values(0).nunique()} tickers")
    return df

raw_df = robust_downloader(TOP_50_TICKERS)

Initiating resilient download for 50 tickers...


[*********************100%***********************]  15 of 15 completed


[*********************100%***********************]  15 of 15 completed


[*********************100%***********************]  15 of 15 completed


[*********************100%***********************]  5 of 5 completed


All tickers downloaded successfully.
✓ Data ingestion complete. File saved: d:\VSCode\Projects\Stock-Market-Trend-Analysis\Data\market_data_raw.csv
✓ Successfully collected 50 tickers


In [47]:
# Add this cell to check your data:
df = pd.read_csv(proc_dir / "market_data_clusters.csv")
print(f"Total rows: {len(df)}")
print(f"Tickers: {df['Ticker'].nunique()}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"\nRows per ticker:")
print(df['Ticker'].value_counts().describe())
print(f"\nMissing values:")
print(df.isnull().sum())

Total rows: 6950
Tickers: 50
Date range: 2025-08-04 to 2026-01-12

Rows per ticker:
count     50.0
mean     139.0
std        0.0
min      139.0
25%      139.0
50%      139.0
75%      139.0
max      139.0
Name: count, dtype: float64

Missing values:
Date              0
Open              0
High              0
Low               0
Close             0
Volume            0
RSI_14            0
SMA_5             0
SMA_20            0
SMA_Ratio_5_20    0
MACD              0
Signal            0
MACD_Histogram    0
BB_Upper          0
BB_Lower          0
BB_Position       0
Vol_SMA_20        0
Volume_Ratio      0
Ticker            0
Anomaly_Flag      0
Is_Anomaly        0
Cluster           0
dtype: int64


In [42]:
current_dir = Path.cwd()
project_root = current_dir if current_dir.name != 'Notebooks' else current_dir.parent
raw_file = project_root / "Data" / "market_data_raw.csv"
proc_dir = project_root / "Data" 
proc_dir.mkdir(parents=True, exist_ok=True)

def calculate_indicators(df):
    """Calculates technical indicators"""
    df = df.copy()  # Avoid SettingWithCopyWarning
    
    # RSI_14
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI_14'] = 100 - (100 / (1 + rs))

    # SMA Ratio 5/20
    df['SMA_5'] = df['Close'].rolling(window=5).mean()
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_Ratio_5_20'] = df['SMA_5'] / df['SMA_20']

    # MACD Histogram
    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26
    df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_Histogram'] = df['MACD'] - df['Signal']

    # Bollinger Bands Position
    std = df['Close'].rolling(window=20).std()
    df['BB_Upper'] = df['SMA_20'] + (std * 2)
    df['BB_Lower'] = df['SMA_20'] - (std * 2)
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])

    # Volume Ratio
    df['Vol_SMA_20'] = df['Volume'].rolling(window=20).mean()
    df['Volume_Ratio'] = df['Volume'] / df['Vol_SMA_20']

    return df.dropna()

print("Loading raw data and calculating features...")
df_raw = pd.read_csv(raw_file, header=[0, 1], index_col=0, parse_dates=True)
tickers = df_raw.columns.get_level_values(0).unique()

all_processed = []
for ticker in tickers:
    try:
        stock_data = df_raw[ticker].copy()
        if stock_data.empty or len(stock_data) < 50:
            print(f"⚠ Skipping {ticker}: Insufficient data ({len(stock_data)} days)")
            continue 

        processed_stock = calculate_indicators(stock_data)
        processed_stock['Ticker'] = ticker  # No more SettingWithCopyWarning
        all_processed.append(processed_stock)
        print(f"✓ Processed {ticker}: {len(processed_stock)} days")
    except Exception as e:
        print(f"✗ Error processing {ticker}: {str(e)[:50]}")

final_df = pd.concat(all_processed)
final_df.to_csv(proc_dir / "market_data_features.csv")
print(f"\n✓ Total: {len(final_df)} rows across {final_df['Ticker'].nunique()} tickers")
print(f"✓ File saved: {proc_dir / 'market_data_features.csv'}")

Loading raw data and calculating features...
✓ Processed AMZN: 139 days
✓ Processed NVDA: 139 days
✓ Processed TSM: 139 days
✓ Processed AVGO: 139 days
✓ Processed LLY: 139 days
✓ Processed AAPL: 139 days
✓ Processed META: 139 days
✓ Processed 2222.SR: 139 days
✓ Processed WMT: 139 days
✓ Processed GOOG: 139 days
✓ Processed TCEHY: 139 days
✓ Processed TSLA: 139 days
✓ Processed JPM: 139 days
✓ Processed MSFT: 139 days
✓ Processed BRK-B: 139 days
✓ Processed BABA: 139 days
✓ Processed 005930.KS: 139 days
✓ Processed ASML: 139 days
✓ Processed PLTR: 139 days
✓ Processed COST: 139 days
✓ Processed ABBV: 139 days
✓ Processed BAC: 139 days
✓ Processed V: 139 days
✓ Processed XOM: 139 days
✓ Processed MC.PA: 139 days
✓ Processed ORCL: 139 days
✓ Processed JNJ: 139 days
✓ Processed NFLX: 139 days
✓ Processed MA: 139 days
✓ Processed 601288.SS: 139 days
✓ Processed GE: 139 days
✓ Processed AMD: 139 days
✓ Processed MU: 139 days
✓ Processed CVX: 139 days
✓ Processed PG: 139 days
✓ Processed CS

In [43]:
anomaly_df = pd.read_csv(proc_dir / "market_data_features.csv")
model_root = project_root / "Models"
model_root.mkdir(parents=True, exist_ok=True)

features = [
    'RSI_14',
    'SMA_Ratio_5_20',
    'MACD_Histogram',
    'BB_Position',
    'Volume_Ratio'
]

clean_anomaly_df = anomaly_df.dropna(subset=features).copy()

iso_model = IsolationForest(n_estimators=100, max_samples='auto', contamination=0.05, random_state=42, max_features=len(features), n_jobs=-1)

clean_anomaly_df['Anomaly_Flag'] = iso_model.fit_predict(clean_anomaly_df[features])
clean_anomaly_df['Is_Anomaly'] = clean_anomaly_df['Anomaly_Flag'].apply(lambda x: 1 if x == -1 else 0)

joblib.dump(iso_model, model_root / "isolation_forest_model.pkl")
clean_anomaly_df.to_csv(proc_dir / "market_data_anomalies.csv", index=False)

print(f"Detected {clean_anomaly_df['Is_Anomaly'].sum()} anomalies across all tickers.")

Detected 348 anomalies across all tickers.


In [44]:
clustering_df = pd.read_csv(proc_dir / "market_data_anomalies.csv")
clean_df = clustering_df.dropna(subset=features).copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(clean_df[features])

optimal_k = 5
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
clean_df['Cluster'] = kmeans.fit_predict(X_scaled)

model_root = project_root / "Models"
model_root.mkdir(parents=True, exist_ok=True)
joblib.dump(kmeans, model_root / "kmeans_model.pkl")
joblib.dump(scaler, model_root / "scaler.pkl")
clean_df.to_csv(proc_dir / "market_data_clusters.csv", index=False)

def find_similar_tickers(query_features_dict, df, kmeans_model, scaler_model, top_n=5):
    query_df = pd.DataFrame([query_features_dict])
    query_scaled = scaler_model.transform(query_df[features])
    
    query_cluster = kmeans_model.predict(query_scaled)[0]
    
    latest_states = df.sort_values('Date').groupby('Ticker').last().reset_index()
    cluster_peers = latest_states[latest_states['Cluster'] == query_cluster].copy()
    
    if cluster_peers.empty:
        return pd.DataFrame()
        
    peer_features_scaled = scaler_model.transform(cluster_peers[features])
    distances = cdist(query_scaled, peer_features_scaled, metric='euclidean')[0]
    
    cluster_peers['Distance'] = distances
    return cluster_peers.nsmallest(top_n, 'Distance')[['Ticker', 'Cluster', 'Distance'] + features]

query_features = {
    'RSI_14': 65.5, 
    'SMA_Ratio_5_20': 1.02, 
    'MACD_Histogram': 0.5,
    'BB_Position': 0.7, 
    'Volume_Ratio': 1.3
}

similar = find_similar_tickers(query_features, clean_df, kmeans, scaler, top_n=3)
print(similar)

     Ticker  Cluster  Distance     RSI_14  SMA_Ratio_5_20  MACD_Histogram  \
3   2222.SR        3  1.330287  65.989853        1.007578        0.106177   
48      WMT        3  1.790001  77.536269        1.019433        0.460009   
39   ROG.SW        3  1.836693  82.063856        1.028796        0.735200   

    BB_Position  Volume_Ratio  
3      1.102233      1.190599  
48     1.230177      1.208467  
39     0.967404      0.742756  


In [45]:
df = pd.read_csv(proc_dir / "market_data_clusters.csv")
df['Target'] = (df.groupby('Ticker')['Close'].shift(-1) > df['Close']).astype(int)
features = [
    'RSI_14', 
    'MACD_Histogram',
    'BB_Position', 
    'SMA_Ratio_5_20',
    'Volume_Ratio', 
    'Is_Anomaly', 
    'Cluster'
]
clean_df = df.dropna(subset=['Target'] + features).copy()
split_idx = int(len(clean_df) * 0.8)
train_df = clean_df.iloc[:split_idx]
test_df = clean_df.iloc[split_idx:]
X_train, y_train = train_df[features], train_df['Target']
X_test, y_test = test_df[features], test_df['Target']

xgb_model = xgb.XGBClassifier(n_estimators=500,learning_rate=0.05,max_depth=5,random_state=42,eval_metric='logloss')
xgb_model.fit(X_train, y_train,eval_set=[(X_test, y_test)],verbose=False)

y_pred = xgb_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Directional Accuracy: {acc:.2%}")

joblib.dump(xgb_model, model_root / "trend_model.pkl")

Directional Accuracy: 52.01%


['d:\\VSCode\\Projects\\Stock-Market-Trend-Analysis\\Models\\trend_model.pkl']

In [46]:
df=pd.read_csv(proc_dir / "market_data_clusters.csv")
df.shape


(6950, 22)